In [5]:
import torch
import torch.nn as nn

In [6]:
input,  H, output = 123, 50, 10

model = nn.Sequential(
    nn.Linear(input, H),
    nn.ReLU(),
    nn.Linear(H, output),
)

In [8]:
model(torch.randn(20,123)).shape

torch.Size([20, 10])